In [24]:
import xarray
import numba as nb
import numpy as np
from numba import njit, prange

In [46]:
xarray.open_zarr("/projects/dgs/persad_research/SIMULATION_DATA/ZARR/LENS2/SIM_VARIABLES/CESM2_historical_day_hurs.zarr")

<xarray.Dataset> Size: 33GB
Dimensions:       (member: 10, time: 14600, lat: 192, lon: 288, ilev: 33,
                   lev: 32, nbnd: 2)
Coordinates:
  * ilev          (ilev) float64 264B 2.255 5.032 10.16 ... 967.5 985.1 1e+03
  * lat           (lat) float64 2kB -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lev           (lev) float64 256B 3.643 7.595 14.36 ... 957.5 976.3 992.6
  * lon           (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
  * member        (member) <U9 360B 'r1i1p1f1' 'r2i1p1f1' ... 'r10i1p1f1'
  * time          (time) object 117kB 1960-01-01 00:00:00 ... 1999-12-31 00:0...
Dimensions without coordinates: nbnd
Data variables: (12/26)
    P0            (member, time) float64 1MB dask.array<chunksize=(1, 14600), meta=np.ndarray>
    RHREFHT       (member, time, lat, lon) float32 32GB dask.array<chunksize=(1, 14600, 48, 72), meta=np.ndarray>
    ch4vmr        (member, time) float64 1MB dask.array<chunksize=(1, 14600), meta=np.ndarray>
    co2vmr        (member, time) float64 1MB dask.array<chunksize=(1, 14600), meta=np.ndarray>
    date          (member, time) float64 1MB dask.array<chunksize=(1, 14600), meta=np.ndarray>
    date_written  (member, time) object 1MB dask.array<chunksize=(5, 7300), meta=np.ndarray>
    ...            ...
    nsbase        (member, time) float64 1MB dask.array<chunksize=(1, 14600), meta=np.ndarray>
    nscur         (member, time) float64 1MB dask.array<chunksize=(1, 14600), meta=np.ndarray>
    nsteph        (member, time) float64 1MB dask.array<chunksize=(1, 14600), meta=np.ndarray>
    sol_tsi       (member, time) float64 1MB dask.array<chunksize=(1, 14600), meta=np.ndarray>
    time_bnds     (member, time, nbnd) object 2MB dask.array<chunksize=(1, 14600, 2), meta=np.ndarray>
    time_written  (member, time) object 1MB dask.array<chunksize=(5, 7300), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    case:              b.e21.BHISTcmip6.f09_g17.LE2-1001.001
    host:              mom1
    initial_file:      b.e21.B1850.f09_g17.CMIP6-piControl.001.cam.i.1001-01-...
    logname:           sunseon
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    source:            CAM
    time_period_freq:  day_1
    topography_file:   /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/fv_0.9x...

In [29]:
%%time

def index_heatwaves(hot_days_ts: np.ndarray, min_duration: int, max_break: int, max_subs: int) -> np.ndarray:
    """
    Identifies the heatwaves in the timeseries using the specified heatwave definition

    Keyword arguments:
    timeseries -- integer array of ones and zeros where ones indicates a hot day (numpy.ndarray)
    max_break -- the maximum number of days between hot days within one heatwave event (default 1)
    min_duration -- the minimum number of hot days to constitute a heatwave event, including after breaks (default 3)
    max_subs -- the maximum number of subsequent events allowed to be apart of the initial consecutive hot days
    """
    ts = np.zeros(hot_days_ts.size + 2, dtype=int)
    for i in range(0, hot_days_ts.size):
        if hot_days_ts[i]:
            ts[i + 1] = 1
    diff_ts = np.diff(ts)
    diff_indices = np.where(diff_ts != 0)[0]

    in_heatwave = False
    current_hw_index = 0
    sub_events = 0
    hw_indices = np.zeros(diff_ts.size, dtype=int)

    for i in range(diff_indices.size-1):
        index = diff_indices[i]
        next_index = diff_indices[i+1]

        if diff_ts[index] == 1 and next_index - index >= min_duration and not in_heatwave:
            current_hw_index += 1
            in_heatwave = True
            hw_indices[index:next_index] = current_hw_index
        elif diff_ts[index] == -1 and next_index - index > max_break:
            in_heatwave = False
        elif diff_ts[index] == 1 and in_heatwave and sub_events < max_subs:
            sub_events += 1
            hw_indices[index:next_index] = current_hw_index
        elif diff_ts[index] == 1 and in_heatwave and sub_events >= max_subs:
            if next_index - index >= min_duration:
                current_hw_index += 1
                hw_indices[index:next_index] = current_hw_index
            else:
                in_heatwave = False
            sub_events = 0

    return hw_indices[0:hw_indices.size-1]

hot_days_ts = (np.random.random(100000000) > 0.5)
index_heatwaves(hot_days_ts, 3, 1, 1)

CPU times: user 32.6 s, sys: 244 ms, total: 32.8 s
Wall time: 32.9 s


array([0, 0, 0, ..., 0, 0, 0])

In [31]:
# %%time

# @njit
# def index_heatwaves(hot_days_ts: np.ndarray, min_duration: int, max_break: int, max_subs: int) -> np.ndarray:
#     """
#     Identifies the heatwaves in the timeseries using the specified heatwave definition

#     Keyword arguments:
#     timeseries -- integer array of ones and zeros where ones indicates a hot day (numpy.ndarray)
#     max_break -- the maximum number of days between hot days within one heatwave event (default 1)
#     min_duration -- the minimum number of hot days to constitute a heatwave event, including after breaks (default 3)
#     max_subs -- the maximum number of subsequent events allowed to be apart of the initial consecutive hot days
#     """
#     ts = np.zeros(hot_days_ts.size + 2, dtype=nb.int64)
#     for i in range(0, hot_days_ts.size):
#         if hot_days_ts[i]:
#             ts[i + 1] = 1
#     diff_ts = np.diff(ts)
#     diff_indices = np.where(diff_ts != 0)[0]

#     in_heatwave = False
#     current_hw_index = 0
#     sub_events = 0
#     hw_indices = np.zeros(diff_ts.size, dtype=nb.int64)

#     for i in range(diff_indices.size-1):
#         index = diff_indices[i]
#         next_index = diff_indices[i+1]

#         if diff_ts[index] == 1 and next_index - index >= min_duration and not in_heatwave:
#             current_hw_index += 1
#             in_heatwave = True
#             hw_indices[index:next_index] = current_hw_index
#         elif diff_ts[index] == -1 and next_index - index > max_break:
#             in_heatwave = False
#         elif diff_ts[index] == 1 and in_heatwave and sub_events < max_subs:
#             sub_events += 1
#             hw_indices[index:next_index] = current_hw_index
#         elif diff_ts[index] == 1 and in_heatwave and sub_events >= max_subs:
#             if next_index - index >= min_duration:
#                 current_hw_index += 1
#                 hw_indices[index:next_index] = current_hw_index
#             else:
#                 in_heatwave = False
#             sub_events = 0

#     return hw_indices[0:hw_indices.size-1]

# hot_days_ts = (np.random.random(100000000) > 0.5)
# index_heatwaves(hot_days_ts, 3, 1, 1)

CPU times: user 1.46 s, sys: 340 ms, total: 1.8 s
Wall time: 1.81 s


array([      0,       0,       0, ..., 5769152, 5769152,       0])

In [35]:
x = 2 * sample / 3 + 5

In [37]:
x.to_netcdf("tmp_checkpoints/calc1_checkpoint.nc")

In [39]:
hot_days_ts

array([False,  True,  True, ...,  True,  True, False])

In [42]:
with open("tmp_checkpoints/hot_days_ts.npy", 'wb') as f:
    np.save(f, hot_days_ts)

In [43]:
reload_hot_days_ts = np.load("tmp_checkpoints/hot_days_ts.npy")

In [44]:
reload_hot_days_ts

array([False,  True,  True, ...,  True,  True, False])